In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Portfolio look-through in LUSID

Shows how to compute the value of a child portfolio's holding as though they were directly held by the parent portfolio.

Attributes
----------
valuations
portfolios
holdings
look through
securitised portfolios
"""

toggle_code("Toggle Docstring")

# Look-through valuation

This notebook shows the portfolio look-through functionality in LUSID. The term "look-through" typically refers to a situation where the holdings of a portfolio are themselves securitised portfolios. "Looking through" the parent portfolio means considering the child portfolio holdings as though they were directly held by the parent.

This is useful for monitoring the performance of underlying holdings, or for ensuring that total exposure to an instrument does not exceed a compliance limit. Without a look-through function, the user would have to aggregate the holdings manually to ensure that there are no breaches.

In the example that follows, we will load holdings for two child portfolios and add them to a parent portfolio. We then demonstrate how to run valuation on the parent portfolio with and without look-through.

Table of contents:
- [1. Setup](#1.-Setup)
- [2. Prepare child portfolios](#2.-Prepare-child-portfolios)
  - [2.1 Load portfolios](#2.1-Load-portfolios)
  - [2.2 Securitise portfolios](#2.2-Securitise-portfolios)
  - [2.3 Load quotes for holdings](#2.3-Load-quotes-for-holdings)
  - [2.4 Load FX rate quotes](#2.4-Load-FX-rate-quotes)
- [3. Create parent portfolio](#3.-Create-parent-portfolio-and-recipes-for-valuation)
- [4. Run valuation on parent portfolio](#4.-Run-valuation-on-parent-portfolio)
    - [4.1 Create valuation recipes](#4.1-Create-valuation-recipes)
    - [4.2 Run valuation](#4.2-Run-valuation)

---

## 1. Setup

Before we can begin, we need to import the required libraries and authenticate on LUSID. For guidance on how to authenticate, see [this page](https://support.lusid.com/knowledgebase/article/KA-01916/en-us).

In [2]:
# Import Libraries
import os
import pandas as pd
import json
import datetime
import pytz

from typing import List, Tuple, Dict
from IPython.core.display import HTML
from datetime import datetime, timedelta

# Import LUSID
import lusid
import lusid.models as models

from lusidtools.cocoon import cocoon
from lusidjam import RefreshingToken
from lusidtools.cocoon.seed_sample_data import seed_data

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format
display(HTML("<style>.container { width:90% !important; }</style>"))

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

api_status = pd.DataFrame(
    api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().to_dict()
)

display(api_status)

,api_version,build_version,excel_version,links
0,v0,0.6.11289.0,0.5.3240,"{'relation': 'RequestLogs', 'href': 'http://ca..."


In [3]:
scope = "fundOfFunds"

child_port_data_filepath = r"data/equity_uk_us_data.csv"
parent_port_data_filepath = r"data/parent_fund_holdings.csv"
mappings_filepath = r"config/upload_config.json"

In [4]:
quotes_api = api_factory.build(lusid.api.QuotesApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
aggregation_api = api_factory.build(lusid.AggregationApi)

## 2. Prepare child portfolios

### 2.1 Load portfolios

We'll start by loading the data for our two child portfolios, one containing UK equities and another containing US equities.

To do so, we need two things: the data itself, and a specification of how to map it onto LUSID's [Data Model](https://www.lusid.com/docs/api/#section/Data-Model). The latter is typically provided via a mappings file that contains information about which identifiers and properties are being loaded for each entity (`Portfolio`, `Instrument`, `Transaction`).

In [5]:
child_portfolio_data = pd.read_csv(child_port_data_filepath)

child_portfolio_data.head(2)

,portfolio_code,portfolio_name,portfolio_base_currency,ticker,sedol,instrument_type,instrument_id,name,txn_id,txn_type,txn_trade_date,txn_settle_date,txn_units,txn_price,txn_consideration,currency,sector,cash_transactions
0,ukEquityPortfolio,UK Active Equity Portfolio,GBP,BT.A,SEDOL1,equity,EQ_1234,BT GROUP PLC,trd_0001,StockIn,02/01/2020,04/01/2020,60000,1,60000,GBP,media,NaN
1,ukEquityPortfolio,UK Active Equity Portfolio,GBP,STAN,SEDOL2,equity,EQ_1235,STANDARD CHARTERED PLC,trd_0002,StockIn,02/01/2020,04/01/2020,100000,1,100000,GBP,financial,NaN


In [6]:
# Process a mapping file for loading data
with open(mappings_filepath) as mappings_file:
    seed_data_mapping = json.load(mappings_file)

In [7]:
# Load the portfolio data using the mappings provided
seed_data_response = seed_data(
    api_factory,
    ["portfolios", "instruments", "transactions"],
    scope,
    child_portfolio_data,
    "DataFrame",
    mappings=seed_data_mapping,
)

### 2.2 Securitise portfolios

In order for the child portfolios to be considered as holdings in a parent portfolio, they must first be securitised. In LUSID, this corresponds to registering them as `Instruments` linked to a portfolio, which we can do using the [Instruments API](https://www.lusid.com/docs/api/#tag/Instruments). We register these instruments with the `SimpleInstrument` instrument type, and we will later instruct LUSID how to treat these instruments using a [Configuration Recipes](https://support.lusid.com/knowledgebase/article/KA-01895).

In [8]:
portfolios = child_portfolio_data["portfolio_code"].unique()

# Creating IDs for the securitised portfolios
securitised_portfolio_ids = [port + "InstrumentCode" for port in portfolios]
securitised_portfolios = list(zip(portfolios, securitised_portfolio_ids))

securitised_portfolios

[('ukEquityPortfolio', 'ukEquityPortfolioInstrumentCode'),
 ('usEquityPortfolio', 'usEquityPortfolioInstrumentCode')]

In [9]:
equity = models.SimpleInstrument(
    instrument_type="SimpleInstrument",
    dom_ccy="USD",
    asset_class="Equities",
    simple_instrument_type="EquityIndex"
)

# Insert the two new instruments into LUSID
for port, instr_id in securitised_portfolios:
    instr_result = instruments_api.upsert_instruments(
        request_body={
            "look_through": models.InstrumentDefinition(
                name=port,
                identifiers={
                    "ClientInternal": models.InstrumentIdValue(value=instr_id)
                },
                definition=equity,
                look_through_portfolio_id=models.ResourceId(scope=scope, code=port),
            )
        }
    )

### 2.3 Load quotes for holdings

Next, we want to generate price quotes for:
- each holding in the child portfolio;
- the securitised child portfolios;
- USD and GBP FX rates, since we have a mix of US and UK equities;

We need these quotes so we can run valuation on the portfolios later.

Quotes are added through calls to the `UpsertQuotes` end-point of the LUSID [Quotes API](https://www.lusid.com/docs/api/#tag/Quotes). For our example, we'll generate dummy price quotes for the first 90 days of 2021, although typically quotes would be sourced from a market data provider.

In [10]:
start_date = datetime(year=2021, month=1, day=1)
num_of_days = 90

# Generate dates for the first 90 days of the current year in YYYY-mm-dd format
days = [
    (start_date + timedelta(days=x)).strftime(format="%Y-%m-%d")
    for x in range(num_of_days)
]

days[:5]

['2021-01-01', '2021-01-02', '2021-01-03', '2021-01-04', '2021-01-05']

In [11]:
def generate_quotes(
    instruments_prices: List[Tuple[str, int, str]],
    instrument_id_type: str,
    quote_type: str,
    pricing_type: str,
) -> Dict:
    """Generates price quotes compatible with the `UpsertQuotes` endpoint.

    Args:
        instruments_prices (List[Tuple[str, int, str]]): list of tuples containing instrument ID, price and currency
        instrument_id_type (str): type of instrument ID to generate a QuoteSeriesId for
        quote_type (str): type of quote to generate
        pricing_type (str): "Static" pricing requests quote with the same price across all 90 days.
        "Dynamic" pricing starts with a base price of 10 and increases it by 5% each day.

    Returns:
        Dict: quotes to upsert
    """

    quotes_for_upsert = {}

    for instr_id, price, currency in instruments_prices:

        if pricing_type == "Static":
            prices = [price] * len(days)
        elif pricing_type == "Dynamic":
            prices = [10 + (x * 0.05) for x in range(num_of_days)]
        else:
            raise ValueError("Unknown pricing type. Should be 'static' or 'dynamic'")

        daily_prices = tuple(zip(days, prices))

        for date, price in daily_prices:

            # Generate the quote
            quotes_for_upsert[
                "quotes_request_" + instr_id + "_" + date.replace("-", "")
            ] = models.UpsertQuoteRequest(
                quote_id=models.QuoteId(
                    quote_series_id=models.QuoteSeriesId(
                        provider="Lusid",
                        instrument_id=instr_id,
                        instrument_id_type=instrument_id_type,
                        quote_type=quote_type,
                        field="mid",
                    ),
                    effective_at=date,
                ),
                metric_value=models.MetricValue(value=price, unit=currency),
            )

    return quotes_for_upsert

In [12]:
quotes_for_upsert = {}
holdings_prices = []

for index, row in child_portfolio_data[["instrument_id", "currency"]].iterrows():
    holdings_prices.append((row["instrument_id"], 1, row["currency"]))

In [13]:
quotes_for_upsert = generate_quotes(
    instruments_prices=holdings_prices,
    instrument_id_type="ClientInternal",
    quote_type="Price",
    pricing_type="Dynamic",
)

# Make API call to upsert the holdings' quotes
quote_response = quotes_api.upsert_quotes(scope=scope, request_body=quotes_for_upsert)

For the securitised portfolios, we use static prices of £1,000 and $1,000, respectively. 

In [14]:
port_prices = [1000, 1000]
port_currencies = ["GBP", "USD"]
child_port_prices = list(zip(securitised_portfolio_ids, port_prices, port_currencies))

child_port_prices

[('ukEquityPortfolioInstrumentCode', 1000, 'GBP'),
 ('usEquityPortfolioInstrumentCode', 1000, 'USD')]

In [15]:
quotes_for_upsert = generate_quotes(
    instruments_prices=child_port_prices,
    instrument_id_type="ClientInternal",
    quote_type="Price",
    pricing_type="Static",
)

# Make API call to upsert the securitised portfolios' quotes
quote_response = quotes_api.upsert_quotes(scope=scope, request_body=quotes_for_upsert)

### 2.4 Load FX rate quotes

We're also going to load in FX rate quotes for GBP/EUR and GBP/USD. This works similarly to the quote generation above, except we load a `Rate` instead of a `Price`.

In [16]:
fx_rates = [1.1, 1.3]
ccy_pairs = ["GBP/EUR", "GBP/USD"]
base_currency = [ccy[:3] for ccy in ccy_pairs]

pairs_rates = list(zip(ccy_pairs, fx_rates, base_currency))

pairs_rates

[('GBP/EUR', 1.1, 'GBP'), ('GBP/USD', 1.3, 'GBP')]

In [17]:
quotes_for_upsert = generate_quotes(
    instruments_prices=pairs_rates,
    instrument_id_type="CurrencyPair",
    quote_type="Price",
    pricing_type="Static",
)

# Make API call to upsert FX rate quotes
quote_response = quotes_api.upsert_quotes(scope=scope, request_body=quotes_for_upsert)

## 3. Create parent portfolio

Having securitised the two child portfolios and added price quotes, let's create the parent portfolio and load the former as holdings.

In [18]:
parent_portfolio_code = "FundOfHedgeFunds"
parent_portfolio_data = pd.read_csv(parent_port_data_filepath)

parent_portfolio_data.head(2)

,portfolio_code,portfolio_name,portfolio_base_currency,ticker,sedol,instrument_type,instrument_id,name,txn_id,txn_type,txn_trade_date,txn_settle_date,txn_units,txn_price,txn_consideration,currency,sector,cash_transactions
0,FundOfHedgeFunds,Fund of fund for Hedge Funds,EUR,ukEquityPortfolioInstrumentCode,ukEquityPortfolioInstrumentCode,equityFund,ukEquityPortfolioInstrumentCode,ukEquityPortfolioInstrumentCode,trd_0001,StockIn,02/01/2020,04/01/2020,1000,1,1000,GBP,funds,NaN
1,FundOfHedgeFunds,Fund of fund for Hedge Funds,EUR,usEquityPortfolioInstrumentCode,usEquityPortfolioInstrumentCode,equityFund,usEquityPortfolioInstrumentCode,usEquityPortfolioInstrumentCode,trd_0002,StockIn,02/01/2020,04/01/2020,1000,1,1000,USD,funds,NaN


In [19]:
seed_data_response = seed_data(
    api_factory,
    ["portfolios", "transactions"],
    scope,
    parent_portfolio_data,
    "DataFrame",
    mappings=seed_data_mapping,
)

## 4. Run valuation on parent portfolio

Now that the data is ready, let's value the parent portfolio. To run valuation, we need a set of holdings with price quotes (which we have) and a valuation recipe.

### 4.1 Create valuation recipes

[Recipes](https://support.lusid.com/knowledgebase/article/KA-01895) are sets of instructions for how the valuation is to be calculated. For more information about their usage in LUSID, see [this article](https://support.lusid.com/knowledgebase/article/KA-01896/en-us) in the documentation.

We'll create one recipe with look-through enabled, and one without. We'll then feed these into a function to generate a valuation request, and have LUSID run the valuation itself. The model rule in the lookthrough-enabled recipe instructs LUSID to price `SimpleInstrument`s with the `SimpleStatic` (i.e. quote lookup) model, and the `Sum` portfolio scaling option causes those instruments to expand themselves into a number of copies of their lookthrough portfolios that preserves the correct total value.

In [20]:
# Create look-through-enabled recipe
lookthrough_config_recipe = models.ConfigurationRecipe(
    scope=scope,
    code="lookthrough",
    market=models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
                key="Quote.*.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Price",
                field="mid",
            ),
            models.MarketDataKeyRule(
                key="FX.CurrencyPair.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Rate",
                field="mid",
            ),
        ],
        options=models.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="ClientInternal",
            default_scope=scope,
            attempt_to_infer_missing_fx=True,
        ),
    ),
    pricing=models.PricingContext(
        options={"AllowPartiallySuccessfulEvaluation": False},
        # toggle look through
        model_rules=[
            models.VendorModelRule(
                supplier="Lusid",
                model_name="SimpleStatic",
                instrument_type="SimpleInstrument",
                model_options=models.IndexModelOptions(portfolio_scaling="Sum", model_options_type="IndexModelOptions"),
            )
        ],
    ),
)

upsert_configuration_recipe_response = (
    configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=models.UpsertRecipeRequest(
            configuration_recipe=lookthrough_config_recipe
        )
    )
)

In [21]:
# Create a non-look-through recipe
non_lookthrough_config_recipe = models.ConfigurationRecipe(
    scope=scope,
    code="no-lookthrough",
    market=models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
                key="Quote.ClientInternal.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Price",
                field="mid",
            ),
            models.MarketDataKeyRule(
                key="FX.CurrencyPair.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Rate",
                field="mid",
            ),
        ],
        options=models.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="ClientInternal",
            default_scope=scope,
            attempt_to_infer_missing_fx=True,
        ),
    ),
    pricing=models.PricingContext(
        options={"AllowPartiallySuccessfulEvaluation": False},
        # toggle look through
        model_rules=[
            models.VendorModelRule(
                supplier="Lusid",
                model_name="SimpleStatic",
                instrument_type="SimpleInstrument",
            )
        ],
    ),
)

upsert_configuration_recipe_response = (
    configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=models.UpsertRecipeRequest(
            configuration_recipe=non_lookthrough_config_recipe
        )
    )
)

### 4.2 Run valuation

In [22]:
def generate_valuation_request(
    valuation_effective_at: datetime, recipe_code: str
) -> lusid.models.valuation_request.ValuationRequest:
    """Generate a valuation request compatible with the `GetValuation` endpoint.

    Args:
        valuation_effective_at (datetime): effective date for the valuation
        recipe_code (string): ID of the recipe to be used in the valuation

    Returns:
        valuation_request (lusid.models.valuation_request.ValuationRequest): a LUSID valuation request
    """
    # Create the valuation request
    valuation_request = models.ValuationRequest(
        recipe_id=models.ResourceId(scope=scope, code=recipe_code),
        metrics=[
            models.AggregateSpec("Instrument/default/Name", "Value"),
            models.AggregateSpec("Valuation/PvInReportCcy", "Proportion"),
            models.AggregateSpec("Valuation/PvInReportCcy", "Sum"),
            models.AggregateSpec("Holding/default/Units", "Sum"),
            models.AggregateSpec("Holding/DomCcy", "Value"),
        ],
        group_by=["Instrument/default/Name"],
        portfolio_entity_ids=[
            models.PortfolioEntityId(scope=scope, code=parent_portfolio_code)
        ],
        valuation_schedule=models.ValuationSchedule(
            effective_at=valuation_effective_at.isoformat()
        ),
        report_currency="EUR"
    )

    return valuation_request

Without look-through, the only holdings in the parent fund are the two securitised child portfolios:

In [23]:
aggregation = aggregation_api.get_valuation(
    valuation_request=generate_valuation_request(
        datetime(year=2021, month=3, day=30, tzinfo=pytz.UTC), "no-lookthrough"
    )
)

pd.DataFrame(aggregation.data)

,Instrument/default/Name,Proportion(Valuation/PvInReportCcy),Sum(Valuation/PvInReportCcy),Sum(Holding/default/Units),Holding/DomCcy
0,ukEquityPortfolio,0.57,"1,100,000.00","1,000.00",GBP
1,usEquityPortfolio,0.43,"846,153.85","1,000.00",USD


With look-through, we can see the holdings of each of the child portfolios:

In [24]:
aggregation = aggregation_api.get_valuation(
    valuation_request=generate_valuation_request(
        datetime(year=2021, month=3, day=30, tzinfo=pytz.UTC), "lookthrough"
    )
)

pd.DataFrame(aggregation.data)

,Instrument/default/Name,Proportion(Valuation/PvInReportCcy),Sum(Valuation/PvInReportCcy),Sum(Holding/default/Units),Holding/DomCcy
0,BT GROUP PLC,0.05,"99,894.89","6,306.50",GBP
1,STANDARD CHARTERED PLC,0.09,"166,491.49","10,510.83",GBP
2,J SAINSBURY PLC,0.06,"116,544.04","7,357.58",GBP
3,BARCLAYS PLC,0.03,"49,947.45","3,153.25",GBP
4,BP PLC,0.07,"133,193.19","8,408.66",GBP
5,GLAXOSMITHKLINE PLC,0.09,"166,491.49","10,510.83",GBP
6,BURBERRY GROUP PLC,0.09,"166,491.49","10,510.83",GBP
7,OCADO GROUP PLC,0.09,"166,491.49","10,510.83",GBP
8,NEXT PLC,0.02,"33,298.30","2,102.17",GBP
9,GBP,0.00,"1,156.19","1,051.08",GBP
